In [2]:
! pip install transformers

  Using cached transformers-4.45.1-py3-none-any.whl (9.9 MB)
  Using cached huggingface_hub-0.25.1-py3-none-any.whl (436 kB)
  Using cached regex-2024.9.11-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (782 kB)
  Using cached safetensors-0.4.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (435 kB)
  Using cached tokenizers-0.20.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.9 MB)

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertModel
from tqdm import tqdm
import argparse
import os
import pandas as pd

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
s3_path="s3://hugging-face-multiclass-textclassification-bucket-1845397/training_data/newsCorpora.csv"
df= pd.read_csv(s3_path, sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER','CATEGORY','STORY', 'HOSTNAME', 'TIMESTAMP'])

df = df[['TITLE', 'CATEGORY']]

my_dict={
        'e': 'Entertainment',
        'b': 'Business',
        't': 'Science',
        'm':'Health'

}

def update_cat(x):
    return my_dict[x]

df['CATEFGORY']=df['CATEGORY'].apply(lambda x: update_cat(x))

print(df)


                                                    TITLE CATEGORY CATEFGORY
0       Fed official says weak data caused by weather,...        b  Business
1       Fed's Charles Plosser sees high bar for change...        b  Business
2       US open: Stocks fall after Fed official hints ...        b  Business
3       Fed risks falling 'behind the curve', Charles ...        b  Business
4       Fed's Plosser: Nasty Weather Has Curbed Job Gr...        b  Business
...                                                   ...      ...       ...
422414  Surgeons to remove 4-year-old's rib to rebuild...        m    Health
422415  Boy to have surgery on esophagus after battery...        m    Health
422416  Child who swallowed battery to have reconstruc...        m    Health
422417  Phoenix boy undergoes surgery to repair throat...        m    Health
422418  Phoenix boy undergoes surgery to repair throat...        m    Health

[422419 rows x 3 columns]


In [5]:
#This is just tip for real life, train on a fraction to make sure the setup is working before training on the whole dataset

# df = df.sample(frac=0.05, random_state=1)

# df=df.reset_index(drop=True)

#This is where the tip ends

In [8]:
df

,TITLE,CATEGORY,CATEFGORY
0,"Fed official says weak data caused by weather,...",b,Business
1,Fed's Charles Plosser sees high bar for change...,b,Business
2,US open: Stocks fall after Fed official hints ...,b,Business
3,"Fed risks falling 'behind the curve', Charles ...",b,Business
4,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,b,Business
...,...,...,...
422414,Surgeons to remove 4-year-old's rib to rebuild...,m,Health
422415,Boy to have surgery on esophagus after battery...,m,Health
422416,Child who swallowed battery to have reconstruc...,m,Health
422417,Phoenix boy undergoes surgery to repair throat...,m,Health


In [12]:
encode_dict ={}

def encode_cat(x):
    if x not in  encode_dict.keys():
        encode_dict[x]=len(encode_dict)
    return encode_dict[x]

In [13]:
df['ENCODE_CAT']=df['CATEGORY'].apply(lambda x:encode_cat(x))

In [14]:
df

,TITLE,CATEGORY,CATEFGORY,ENCODE_CAT
0,"Fed official says weak data caused by weather,...",b,Business,0
1,Fed's Charles Plosser sees high bar for change...,b,Business,0
2,US open: Stocks fall after Fed official hints ...,b,Business,0
3,"Fed risks falling 'behind the curve', Charles ...",b,Business,0
4,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,b,Business,0
...,...,...,...,...
422414,Surgeons to remove 4-year-old's rib to rebuild...,m,Health,3
422415,Boy to have surgery on esophagus after battery...,m,Health,3
422416,Child who swallowed battery to have reconstruc...,m,Health,3
422417,Phoenix boy undergoes surgery to repair throat...,m,Health,3


In [15]:
pip show transformers

Name: transformers
Version: 4.45.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /opt/conda/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [16]:
#!pip install transformers ==4.41.1

In [19]:
from transformers import DistilBertTokenizer

tokenizer=DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
inputs=tokenizer.encode_plus(
                'I love Jesus, He is my Lord', 
                "Jesus loves me",
                add_special_tokens=True,
                max_length=20,
                padding = 'max_length', 
                truncation=True, 
                return_token_type_ids=True,
                return_attention_mask=True


)

print("Inputs IDs:", inputs['input_ids'])
print("Attention Masks:", inputs['attention_mask'])
print("Token type IDs:", inputs['token_type_ids'])

Inputs IDs: [101, 1045, 2293, 4441, 1010, 2002, 2003, 2026, 2935, 102, 4441, 7459, 2033, 102, 0, 0, 0, 0, 0, 0]
Attention Masks: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
Token type IDs: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
